# Function for Pdf Text Reading:

In [104]:
import re
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text_per_page = []
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            text_per_page.append(text)
    return text_per_page

if __name__ == "__main__":
    pdf_path = r'C:\Users\Awan\Desktop\New folder\Statement 23-24\Sch Issued 1st Aug 23 will be paid 1st Sept 23\3954630007.pdf'
    extracted_text = extract_text_from_pdf(pdf_path)
    
    # Extract statement reference
    statement_reference = None
    for text in extracted_text:
        if statement_reference is None:
            statement_reference_match = re.search(r'Statement\s*Reference\s*([\d/]+)', text)
            if statement_reference_match:
                statement_reference = statement_reference_match.group(1)
                break
    
    if statement_reference:
        print("Extracted Statement Reference:", statement_reference)
    else:
        print("No Statement Reference found in the PDF.")
    
    # Extract the contract details
    text_per_page = []
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            text_per_page.append(text)
            if "Contract Monthly Pay Statement" in text:
                break

    found_start = False
    contract = ""
    for page_num, text in enumerate(text_per_page, start=1):
        if found_start:
            if "CONTRACT" in text:
                break
            contract += text.strip() + "\n"
        elif "CONTRACT" in text:
            found_start = True
            start_index = text.find("CONTRACT") + len("CONTRACT")
            end_index = text.find("If ")
            contract += text[start_index:end_index].strip() + "\n"

    # Split the contract string into Contract_ID and Contract_Name
    contract_lines = contract.split("\n", 1)

    # Assuming the first part is Contract_ID and the second part is Contract_Name
    if len(contract_lines) >= 2:
        Contract_ID = contract_lines[0].strip()
        Contract_Name = contract_lines[1].strip()

        # Print the results
        print("Contract_ID:", Contract_ID)
    else:
        print("Failed to extract contract details properly.")


Extracted Statement Reference: 2324/04
Contract_ID: QRV   3954630007


# Superannuation:

In [108]:
performers_superannuation_table_text

"£116.80\n364207 AJ ABULHAB @ 11.60% £4.75\n114847 Coughlan @ 7.70% £3.80\n651133 S GHAURI @ 5.10% £0.23\n887048 I HAMDY ABD ELRAZEK @ 5.00% (2223) -£11.80\n887048 I HAMDY ABD ELRAZEK @ 5.10% (2223) -£12.90\n887048 I HAMDY ABD ELRAZEK @ 5.10% -£4.12\n786616 AK JAISINGHANI @ 9.80% £0.82\n453390 JS MALIK @ 5.10% £4.30\n107389 ZM MIRZA @ 5.10% (2223) -£0.18\n107389 ZM MIRZA @ 5.10% £0.16\n476617 AD MISTRY @ 6.80% £2.84\n111766 Ja Malik @ 9.30% (2223) -£79.10\n111766 Ja Malik @ 9.80% (2223) -£45.50\n \nTHIS STATEMENT SHOULD BE RETAINED FOR TAX PURPOSES\n2\nNATIONAL HEALTH SERVICE   \n Contract QRV   3954630007\nStatement Reference  2324/04  ASHIQ GHAURI AND SARFRAZ\nGHAURI T/A GHAURI DENTAL\nCENTRE\n   \nDate  01/08/23 VAT No. 654434729\nPAY STATEMENT\nNon Contributory Items Payments Deductions\n111766 Ja Malik @ 9.80% £18.54\n623504 JS UBHI @ 6.10% £4.21\n115616 NZ ZIA @ 9.80% (2223) -£1.00\n115616 NZ ZIA @ 9.80% -£1.85\nPerformers' Added Benefit £0.75\n786616 AK JAISINGHANI @ 9.00% £0.75

In [110]:
import re
import pandas as pd
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text_per_page = []
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            text_per_page.append(text)
    return text_per_page



def extract_performers_superannuation_table(pdf_text):
    # Define the start and end patterns for the table
    start_pattern = r'Performers\' Superannuation Contribution'
    end_pattern = r'Patients\' Charges'

    # Find the start and end positions of the table
    start_match = re.search(start_pattern, pdf_text)
    end_match = re.search(end_pattern, pdf_text)

    # If both start and end patterns are found, extract the table text
    if start_match and end_match:
        table_text = pdf_text[start_match.end():end_match.start()].strip()
        return table_text
    else:
        return None

def format_table(table_text):
    # Extracting rows using regular expression
    #rows = re.findall(r'(\d+)\s+([A-Z\s]+)@?\s+([\d.]+)%\s+£([\d.]+)', table_text)
    
    
    rows = re.findall(r'(\d+)\s+([A-Za-z\s]+)@?\s+([\d.]+)%\s+£([\d.]+)', table_text)
    

    # Formatting the rows into desired format
    formatted_rows = []
    for row in rows:
        formatted_row = {
            "Performer_ID": row[0],
            "Name": row[1],
            "Percentage": row[2],
            "Amount ": row[3]
        }
        formatted_rows.append(formatted_row)

    return formatted_rows

if __name__ == "__main__":
    #pdf_path = 'BSA Statement 01 Highlighted.pdf'
    # Extract statement references from PDF
    extracted_text = extract_text_from_pdf(pdf_path)
    # Extract the table text
    extracted_text = extract_text_from_pdf(pdf_path)
    complete_pdf_text = "\n".join(extracted_text)
    performers_superannuation_table_text = extract_performers_superannuation_table(complete_pdf_text)

    # Print the extracted table text
    if performers_superannuation_table_text:
        print("Performers' Superannuation Contribution Table:")
        formatted_table = format_table(performers_superannuation_table_text)
        # Convert the formatted table to DataFrame
        df2 = pd.DataFrame(formatted_table)
        print(df2)
    else:
        print("Performers' Superannuation Contribution Table not found in the PDF text.")


Performers' Superannuation Contribution Table:
  Performer_ID             Name Percentage Amount 
0       364207      AJ ABULHAB       11.60    4.75
1       114847        Coughlan        7.70    3.80
2       651133        S GHAURI        5.10    0.23
3       786616  AK JAISINGHANI        9.80    0.82
4       453390        JS MALIK        5.10    4.30
5       107389        ZM MIRZA        5.10    0.16
6       476617       AD MISTRY        6.80    2.84
7       111766        Ja Malik        9.80   18.54
8       623504         JS UBHI        6.10    4.21
9       786616  AK JAISINGHANI        9.00    0.75


In [96]:

cleaned_table_text = re.sub(r'\(\d+\)', '', performers_superannuation_table_text)

In [98]:
cleaned_table_text

'£9,586.67\n107793 SA ALI @ 5.00%  -£71.94\n107793 SA ALI @ 5.70%  -£68.60\n107793 SA ALI @ 5.70% -£81.98\n105483 BAGHERI @ 9.30%  £463.79\n105483 BAGHERI @ 9.80%  £485.73\n105483 BAGHERI @ 9.80% £348.81\n651133 S GHAURI @ 5.10% £0.23\n107884 HAQ @ 12.50% -£83.04\n705276 Z MUSHTAQ @ 13.50% -£39.11\n113245 QUERESHI @ 11.60%  £3,019.73\n113245 QUERESHI @ 12.50%  £3,255.57\n113245 QUERESHI @ 11.60% £2,015.81\n101949 ZAMAN @ 9.80% £341.67'

In [77]:
cleaned_table_text = re.sub(r'\(\d+\)', '', performers_superannuation_table_text)


# Define the pattern to identify and remove the unwanted section
pattern = r'Performers\' Added Benefit.*$'

# Use regex to substitute the matched pattern with an empty string
cleaned_text = re.sub(pattern, '', cleaned_table_text, flags=re.DOTALL)

# Trim any leading or trailing whitespace
cleaned_text = cleaned_text.strip()

In [52]:
cleaned_text

'£31.22\n364207 AJ ABULHAB @ 11.60% £0.02\n798878 HK BHINDER @ 9.80% £9.98\n112649 GUPTA @ 9.80% £4.37\n102657 IOANNIDIS @ 11.60% £0.01\n402257 UA MALIK @ 11.60% £15.54\n623504 JS UBHI @ 5.10% £1.30'

In [105]:
import re
import pandas as pd

import re
import pandas as pd

def extract_performers_superannuation_table(pdf_text):
    # Define the start and end patterns for the table
    start_pattern = r'Performers\' Superannuation Contribution'
    end_pattern = r'Patients\' Charges'

    # Find the start and end positions of the table
    start_match = re.search(start_pattern, pdf_text)
    end_match = re.search(end_pattern, pdf_text)

    # If both start and end patterns are found, extract the table text
    if start_match and end_match:
        table_text = pdf_text[start_match.end():end_match.start()].strip()
        return table_text
    else:
        return None
def extract_rows_from_text(text):
    # Define a more flexible pattern to capture rows
    pattern = r'(\d+)\s+([A-Za-z\s]+)@?\s+([\d.]+)%\s*(-?£[\d.,]+)'
    #rows = re.findall(r'(\d+)\s+([A-Za-z\s]+)@?\s+([\d.]+)%\s+£([\d.]+)', cleaned_table_text)
    
    # Find all matches in the text
    matches = re.findall(pattern, cleaned_text)
    
    # Return a list of matched rows
    return matches

def format_table(rows, Contract_ID, statement_reference):
    formatted_rows = []
    for row in rows:
               # Extract the numeric part of the amount string and convert to float
        amount_str = re.sub(r'[^.\d]', '', row[3])  # Remove all non-numeric characters except '.'
        amount = float(amount_str)  # Convert to float
        
        formatted_row = {
            "contract_id": Contract_ID,
            "statement_reference": statement_reference,
            "performer_id": row[0],
            "name": row[1].strip(),
            "percentage": row[2],
            "amount": amount
        }
        formatted_rows.append(formatted_row)
    return formatted_rows

if __name__ == "__main__":
    
    # Extract rows using the updated pattern
    rows = extract_rows_from_text(cleaned_text)

    
    # Format the extracted rows into DataFrame
    formatted_table = format_table(rows, Contract_ID, statement_reference)
    df2 = pd.DataFrame(formatted_table)
    
    # Display the DataFrame
    print(df2)



        contract_id statement_reference performer_id                 name  \
0  QRV   3954630007             2324/04       887048  I HAMDY ABD ELRAZEK   
1  QRV   3954630007             2324/04       887048  I HAMDY ABD ELRAZEK   
2  QRV   3954630007             2324/04       887048  I HAMDY ABD ELRAZEK   
3  QRV   3954630007             2324/04       107389             ZM MIRZA   
4  QRV   3954630007             2324/04       107389             ZM MIRZA   
5  QRV   3954630007             2324/04       115616               NZ ZIA   
6  QRV   3954630007             2324/04       115616               NZ ZIA   

  percentage  amount  
0       5.00  188.77  
1       5.10  206.33  
2       5.10   65.85  
3       5.10    2.77  
4       5.10    2.54  
5       9.80    0.40  
6       9.80    0.74  


NameError: name 'table_text' is not defined

In [ ]:
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import re
from bs4 import BeautifulSoup
import os
import requests
from selenium.webdriver.chrome.options import Options
import pdfplumber
import pyautogui


import urllib.request


# Path to ChromeDriver executable
chrome_driver_path = "C:\\Users\\Awan\\Downloads\\chromedriver-win64\\chromedriver.exe"  # Adjust this path accordingly


# URL of the login page
url = "https://compass.nhsbsa.nhs.uk/auth/esr.elogin"

# Create a Selenium WebDriver instance
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Navigate to the login page
driver.get(url)

# Wait for the page to load
time.sleep(2)

#memorable_word_text= "sfDceyhj3eLd" 
memorable_word_text= compass_memorableword
# Find the Memorable word character inputs
memorable_word_positions = []
memorable_word_inputs = driver.find_elements(By.CLASS_NAME, "memorable_word")
for input_element in memorable_word_inputs:
    # Extract the position using regular expressions
    aria_label = input_element.get_attribute("aria-label")
    position_match = re.search(r"Enter (\d+)st|(\d+)nd|(\d+)rd|(\d+)th character", aria_label)
    if position_match:
        position = int(next(filter(None, position_match.groups())))
        memorable_word_positions.append(position)
print(memorable_word_positions)

# Find username, password, and memorable word fields and fill them
#driver.find_element(By.ID, "logon.username").send_keys("rghauri")
#driver.find_element(By.ID, "logon.password").send_keys("sdrF35^&$sH")
driver.find_element(By.ID, "logon.username").send_keys(compass_username)
driver.find_element(By.ID, "logon.password").send_keys(compass_password)
driver.find_element(By.ID, "memorable_word_one").send_keys(memorable_word_text[memorable_word_positions[0]-1])
driver.find_element(By.ID, "memorable_word_two").send_keys(memorable_word_text[memorable_word_positions[1]-1])
driver.find_element(By.ID, "memorable_word_three").send_keys(memorable_word_text[memorable_word_positions[2]-1])

# Submit the form
driver.find_element(By.CLASS_NAME, "go_button").click()

# Wait for the login process to complete
time.sleep(5)

# Click on the "Payments" element
payments_element = driver.find_element(By.XPATH, "//div[@name='menu_option_6']")
payments_element.click()

# Wait for the "Payments" page to load
time.sleep(2)

# Click on the "Monthly Statements" link
monthly_statements_element = driver.find_element(By.XPATH, "//div[@name='menu_option_2']")
monthly_statements_element.click()

# Wait for the "Monthly Statements" page to load
time.sleep(2)


contract_ids = [
    "1001710000",
    "1016580000",
    "1016590000",
    "3954630001",
    "3954630002",
    "3954630004",
    "3954630006",
    "3954630007",
    "3954630008",
    "3954630009"
]

for contract_id in contract_ids:
    # Add number into Contract ID field
    contract_id_field = driver.find_element(By.ID, "contract_id")
    contract_id_field.clear()
    contract_id_field.send_keys(contract_id)

    # Click on the "Search" button
    search_button = driver.find_element(By.ID, "search")
    search_button.click()

    # Wait for the search results to load
    time.sleep(2)

    # Get the page source after search button click
    page_source = driver.page_source

    # Parse the page source using BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract all script tags
    script_tags = soup.find_all('script')

    # Get the content of the last script tag
    last_script_tag = script_tags[-1].text
    # Define the regular expression pattern to extract the desired data
    pattern = r"esr_ProcessPageLoad\('(\w+)', '(\w+)', '(\w+)', (true|false)\);"
    # Search for the pattern in the script content
    match = re.search(pattern, last_script_tag)
    data = []
    # Extract the matched groups
    if match:
        data = match.groups()
        data = list(data)

    # Click on the "Search" button
    search_button = driver.find_element(By.ID, "view_" + data[2] + "_main_doc_grid_0")
    search_button.click()

    time.sleep(2)

    # Switch to the new tab
    driver.switch_to.window(driver.window_handles[-1])

    # Wait for a brief moment before starting, to give time to switch to the desired window
    time.sleep(2)

    # Press Ctrl + S
    pyautogui.hotkey('ctrl', 's')

    # Wait for a moment to ensure the save dialog appears
    time.sleep(1)

    # Press Enter
    pyautogui.press('enter')

    # Switch back to the original window
    driver.switch_to.window(driver.window_handles[0])

    # Wait for some time to ensure the original window has loaded properly
    time.sleep(5)

    if "Welcome" in driver.page_source:
        print("Login successful!")
    else:
        print("Login failed.")

# Close the browser
driver.quit()


time.sleep(30)
if "Welcome" in driver.page_source:
    print("Login successful!")
else:
    print("Login failed.")

# Close the browser
driver.quit()
